## Building a Conversational Agent with Function Calling

* **Created by:** Matthew Gault
* **For:** CSCI 3351
* **At:** University of Texas Rio-Grande Valley

## Setup

In [34]:
%pip install -U --quiet pydantic openai gradio geocoder

Note: you may need to restart the kernel to use updated packages.


## Database

In [2]:
from datetime import datetime
from datetime import timedelta
import sqlite3

def create_database():
    # Connect to SQLite database (or create it if it doesn't exist)
    conn = sqlite3.connect('schedule.db')
    cursor = conn.cursor()

    # Create tables
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS users (
        user_id INTEGER PRIMARY KEY AUTOINCREMENT,
        full_name TEXT NOT NULL,
        phone TEXT NOT NULL UNIQUE
    );
    ''')

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS checkup_slots (
        slot_id INTEGER PRIMARY KEY AUTOINCREMENT,
        checkup_date TEXT NOT NULL,
        checkup_time TEXT NOT NULL
    );
    ''')

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS scheduled_checkups (
        checkup_id INTEGER PRIMARY KEY AUTOINCREMENT,
        user_id INTEGER,
        slot_id INTEGER,
        walk_in BOOLEAN,
        appointment BOOLEAN,
        FOREIGN KEY(user_id) REFERENCES users(user_id),
        FOREIGN KEY(slot_id) REFERENCES checkup_slots(slot_id)
    );
    ''')

    # Commit changes
    conn.commit()
    
    # Generate and insert checkup slots
    generate_checkup_slots(cursor)
    
    # Commit changes and close the connection
    conn.commit()
    conn.close()

    print("Database and tables created successfully, and initial data inserted.")

def generate_checkup_slots(cursor):
    # Define the hours and days you want to create slots for
    hours = ["08:00am", "09:00am", "10:00am", "11:00am", "12:00pm", "01:00pm", "02:00pm", "03:00pm", "04:00pm", "05:00pm"]
    days_of_week = [0, 1, 2, 3, 4, 5, 6]  # 0=Monday, 1=Tuesday, ..., 6=Sunday

    start_date = datetime.now()
    end_date = start_date + timedelta(days=365)  # Slots for the next year

    current_date = start_date
    while current_date <= end_date:
        if current_date.weekday() in days_of_week:
            for hour in hours:
                checkup_date = current_date.strftime("%m/%d/%Y")
                cursor.execute('INSERT INTO checkup_slots (checkup_date, checkup_time) VALUES (?, ?)', (checkup_date, hour))
        current_date += timedelta(days=1)
        
create_database()

Database and tables created successfully, and initial data inserted.


In [3]:
import sqlite3
import re
import geocoder

def connect_db():
    return sqlite3.connect('schedule.db')

def format_phone_number(phone):
    # Remove all non-digit characters
    digits = re.sub(r'\D', '', phone)

    # Check if we have exactly 10 digits (U.S. phone number without country code)
    if len(digits) == 10:
        # Reformat to (XXX) XXX-XXXX
        formatted_phone = f"({digits[0:3]}) {digits[3:6]}-{digits[6:10]}"
        return True, formatted_phone
    return False, None

def validate_phone_number(phone):
    pattern = re.compile(r"^\(\d{3}\) \d{3}-\d{4}$")
    return pattern.match(phone) is not None

def validate_date(date_text):
    try:
        datetime.strptime(date_text, "%m/%d/%Y")
        return True
    except ValueError:
        return False

def validate_address(address):
    # Use the geocoder to get the location data
    g = geocoder.osm(address)
    print(g)
    print(g.city)
    if g.ok:
        # Check if the address is in Brownsville, TX
        if 'Brownsville' in g.city and 'Texas' in g.state:
            return True, g.latlng
        else:
            return False, None
    return False, None


def is_slot_available(slot_id):
    conn = connect_db()
    cursor = conn.cursor()
    cursor.execute('SELECT slot_id FROM scheduled_checkups WHERE slot_id = ?', (slot_id,))
    result = cursor.fetchone()
    conn.close()
    return result is None

def schedule_checkup(full_name: str, phone: str, address: str, checkup_slot_id: int, walk_in: bool, appointment: bool):
    "Attempts to schedule a checkup for a user at the desired checkup_slot_id. All fields are required."

    valid_phone, formatted_phone = format_phone_number(phone)
    if not valid_phone:
        return "Invalid phone number. Please provide a valid 10-digit phone number."

    phone = formatted_phone  # Use the formatted phone number

    if not is_slot_available(checkup_slot_id):
        return "This slot is already booked. Please choose another slot."

    valid_address, coordinates = validate_address(address)
    if not valid_address:
        return "Invalid address or not located in Brownsville, TX."

    conn = connect_db()
    cursor = conn.cursor()
    
    try:
        # Insert user if not exists
        cursor.execute('INSERT OR IGNORE INTO users (full_name, phone) VALUES (?, ?)', (full_name, phone))
        user_id = cursor.lastrowid
        
        if user_id == 0:
            cursor.execute('SELECT user_id FROM users WHERE phone = ?', (phone,))
            user_id = cursor.fetchone()[0]

        # Schedule the checkup
        cursor.execute('''
            INSERT INTO scheduled_checkups (user_id, slot_id, walk_in, appointment)
            VALUES (?, ?, ?, ?)
        ''', (user_id, checkup_slot_id, walk_in, appointment))
        
        conn.commit()
    except sqlite3.Error as e:
        conn.rollback()
        conn.close()
        return f"An error occurred: {e}"
    finally:
        conn.close()

    return "Successfully scheduled."

def get_availability_admin():
    "Returns all availability"
    conn = connect_db()
    cursor = conn.cursor()
    
    cursor.execute('SELECT slot_id, checkup_date, checkup_time FROM checkup_slots')
    availability = cursor.fetchall()
    
    conn.close()
    
    return [{"checkup_slot_id": slot_id, "checkup_date": date, "checkup_time": time} for slot_id, date, time in availability]

def get_booked_appointments():
    "Attempts to schedule a checkup for a user at the desired checkup_slot_id."
    conn = connect_db()
    cursor = conn.cursor()
    
    # SQL query to fetch all booked appointments
    cursor.execute('''
        SELECT sp.checkup_id, u.full_name, u.phone, ps.checkup_date, ps.checkup_time, sp.walk_in, sp.appointment
        FROM scheduled_checkups sp
        JOIN users u ON sp.user_id = u.user_id
        JOIN checkup_slots ps ON sp.slot_id = ps.slot_id
        ORDER BY ps.checkup_date, ps.checkup_time
    ''')
    
    # Fetch all results
    appointments = cursor.fetchall()
    conn.close()
    
    # Convert to a list of dictionaries for easier DataFrame conversion
    columns = ["checkup_id", "full_name", "phone", "checkup_date", "checkup_time", "walk_in", "appointment"]
    appointments_data = [dict(zip(columns, appointment)) for appointment in appointments]
    
    return appointments_data


def get_availability():
    "Returns available slots for today and tomorrow"
    conn = connect_db()
    cursor = conn.cursor()
    
    # Get today's and tomorrow's dates in the format stored in the database
    today = datetime.now().strftime("%m/%d/%Y")
    tomorrow = (datetime.now() + timedelta(days=1)).strftime("%m/%d/%Y")
    
    # SQL query to fetch slots for today and tomorrow that are not booked
    cursor.execute('''
        SELECT ps.slot_id, ps.checkup_date, ps.checkup_time
        FROM checkup_slots ps
        LEFT JOIN scheduled_checkups sp ON ps.slot_id = sp.slot_id
        WHERE ps.checkup_date IN (?, ?) AND sp.slot_id IS NULL
        ORDER BY ps.checkup_date, ps.checkup_time
    ''', (today, tomorrow))
    
    availability = cursor.fetchall()
    conn.close()
    
    # Format the results into a list of dictionaries
    return [{"checkup_slot_id": slot_id, "checkup_date": date, "checkup_time": time} for slot_id, date, time in availability]

# Example usage
print(get_booked_appointments())

[]


## Define Tools

In [4]:
def sum(a:int, b:int=1):
    "Adds a + b"
    return a + b

print(sum(a=2, b=3))

5


In [5]:
import requests

def geocode(city_name:str):
    "Geocodes a city name into latitude and longitude data"
    url = f'https://geocoding-api.open-meteo.com/v1/search?name={city_name}&count=10&language=en&format=json'

    try:
        response = requests.get(url)

        # If the response was successful, no Exception will be raised
        response.raise_for_status()
    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')  
    except Exception as err:
        print(f'Other error occurred: {err}')  
    else:
        return response.json() # If successful, return json response
    
print(geocode(city_name="Brownsville"))

{'results': [{'id': 4676740, 'name': 'Brownsville', 'latitude': 25.90175, 'longitude': -97.49748, 'elevation': 10.0, 'feature_code': 'PPLA2', 'country_code': 'US', 'admin1_id': 4736286, 'admin2_id': 4678364, 'timezone': 'America/Chicago', 'population': 183887, 'postcodes': ['78520', '78521', '78522', '78523', '78526'], 'country_id': 6252001, 'country': 'United States', 'admin1': 'Texas', 'admin2': 'Cameron'}, {'id': 4609116, 'name': 'Brownsville', 'latitude': 35.59397, 'longitude': -89.26229, 'elevation': 119.0, 'feature_code': 'PPLA2', 'country_code': 'US', 'admin1_id': 4662168, 'admin2_id': 4628456, 'timezone': 'America/Chicago', 'population': 9876, 'postcodes': ['38012'], 'country_id': 6252001, 'country': 'United States', 'admin1': 'Tennessee', 'admin2': 'Haywood'}, {'id': 4285708, 'name': 'Brownsville', 'latitude': 37.19255, 'longitude': -86.26775, 'elevation': 166.0, 'feature_code': 'PPLA2', 'country_code': 'US', 'admin1_id': 6254925, 'admin2_id': 4290885, 'timezone': 'America/Chi

In [6]:
def weather(latitude:float, longitude:float):
    "Returns the weather conditions for a given latitude and longitude"
    url = f'https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,is_day,precipitation,rain,showers,snowfall&timezone=America%2FChicago'

    try:
        response = requests.get(url)

        # If the response was successful, no Exception will be raised
        response.raise_for_status()
    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')  
    except Exception as err:
        print(f'Other error occurred: {err}')  
    else:
        return response.json() # If successful, return json response
    
print(weather(latitude=26.30174, longitude=-98.16334))

{'latitude': 26.309208, 'longitude': -98.153625, 'generationtime_ms': 0.034928321838378906, 'utc_offset_seconds': -18000, 'timezone': 'America/Chicago', 'timezone_abbreviation': 'CDT', 'elevation': 30.0, 'current_units': {'time': 'iso8601', 'interval': 'seconds', 'temperature_2m': '°C', 'is_day': '', 'precipitation': 'mm', 'rain': 'mm', 'showers': 'mm', 'snowfall': 'cm'}, 'current': {'time': '2024-05-08T14:30', 'interval': 900, 'temperature_2m': 37.2, 'is_day': 1, 'precipitation': 0.0, 'rain': 0.0, 'showers': 0.0, 'snowfall': 0.0}}


In [15]:
from datetime import datetime
from datetime import timedelta

# gets todays date
def get_todays_date():
    "Gets today's date"
    return datetime.now().strftime('%B %d, %Y')

# Example:
print(get_todays_date())

May 08, 2024


In [16]:
# gets tomorrows date
def get_tomorrows_date():
    "Gets tomorrow's date"
    now = datetime.now()
    tomorrow = now + timedelta(days=1)
    return tomorrow.strftime('%B %d, %Y')

print(get_tomorrows_date())

May 09, 2024


In [17]:
# gets the current time
def get_current_time():
    "Gets the current time"
    return datetime.now().strftime("%m/%d/%Y %I:%M %p")

# Example:
print(get_current_time())

05/08/2024 02:44 PM


In [7]:
import geocoder

# finds your city and state based on your IP address
def user_location():
    "Guesses the users current location in the format of <city>, <state> based on IP address. But should always ask the user when scheduling."
    g = geocoder.ip('me')
    city = g.city
    state = g.state
    return f"{city}, {state}"

# Example:
print(user_location())

Brownsville, Texas


## Format for OpenAI

In [8]:
from pydantic import create_model
import inspect, json
from inspect import Parameter

def schema(f):
    kw = {n: (o.annotation, ... if o.default == Parameter.empty else o.default)
          for n, o in inspect.signature(f).parameters.items()}
    s = create_model(f'Input for `{f.__name__}`', **kw).model_json_schema()
    function_schema = dict(name=f.__name__, description=f.__doc__, parameters=s)
    return {
        "type": "function",
        "function": function_schema
    }

def call_func(name, arguments):
    f = globals()[name]
    return f(**json.loads(arguments))

In [9]:
schema(sum)

{'type': 'function',
 'function': {'name': 'sum',
  'description': 'Adds a + b',
  'parameters': {'properties': {'a': {'title': 'A', 'type': 'integer'},
    'b': {'default': 1, 'title': 'B', 'type': 'integer'}},
   'required': ['a'],
   'title': 'Input for `sum`',
   'type': 'object'}}}

In [10]:
call_func("sum", '{"a": 2, "b": 3}')

5

In [11]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)  # take environment variables from .env.

from openai import OpenAI

client = OpenAI(
    base_url=os.getenv("OPENAI_API_BASE"),
    api_key=os.getenv("OPENAI_API_KEY")
)

def chat_completion(
    message,
    model="gpt-4",
    prompt="You are a helpful assistant.",
    temperature=0,
    messages=[],
    tools=[]
):
    # Add the prompt to the messages list
    if prompt is not None:
        messages = [{"role": "system", "content": prompt}] + messages

    if message is not None:
        # Add the user's message to the messages list
        messages += [{"role": "user", "content": message}]

    # NEW
    if tools:
        tool_schemas = [schema(f) for f in tools]
    else:
        tool_schemas = None
        
    # Make an API call to the OpenAI ChatCompletion endpoint with the model and messages
    print(f"messages: {messages}")
    completion = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        tools=tool_schemas
    )
    
    # Updated
    assistant_message = completion.choices[0].message
        
    return assistant_message

In [12]:
chat_completion("Add 2+3", model="gpt-4", tools=[sum])

messages: [{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': 'Add 2+3'}]


ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_33o4Q8dwosb40iYmqdBvGwxH', function=Function(arguments='{\n  "a": 2,\n  "b": 3\n}', name='sum'), type='function')])

In [18]:
# Define a function to handle the chat interaction with the AI model
def chat(message, chatbot_messages, history_state):
    prompt = """
    You are a helpful AI assistant named Bob created by Valley Day & Night Clinic in Brownsville, TX.

    Today's date: {todays_date}
    Current time: {current_time}
    
    ## Task
    
    Your job is to provide an amazing customer service experience for potential customers of Day & Night Clinic and
    current customers.
    
    Additionally, you are renowned for converting leads into customers.
    
    As an AI assistant, you will help faciliate 'check-ups' for Valley Day & Night Clinic's services.
    
    ## Check-Up Services
    
    Standard - copay of $40 per person
    
    A $100 charge will be deducted upon not showing up to the walk-in/appointment at all, so let the customer know about this detail.
    
    ## Walk-ins
    
    Any person can choose to walk into the clinic during our work hours from 8 A.M. to 5 P.M. and pay the $40 copay.
    
    ## Prices
    
    Prices for a standard check-up are $40 as a copay per person admitted into the clinic, regardless of whether it's an 
    appointment or a walk-in.
    
    ## Appointments
    
    If a customer expresses intent for an appointment what they mean is that they would like to schedule a check-up for the Check-Up Services.
        
    Before getting down any of their details, you should always check availability and provide it to the customer.
    
    They don't have a user interface or anything to indicate availability and are relying on you to tell them.
    
    When displaying availability do not display slot IDs as that maps to our internal database IDs and is SENSITIVE.
    
    ## Checkup Scheduling Process
    
    Incrementally ask the customer for the correct information
    
    1. Double check that the service they selected is appointment.
    2. Show them the available appointment slots and identify the one they are interested in.
    3. What their name and phone number is.
    4. Schedule the appointment.
    
    It is important that you break this up into smaller interactions to not overwhelm the user.
    
    Getting their name and phone number before scheduling the appointment is important in case the session gets disconnected.
    """.format(
        todays_date=get_todays_date(),
        current_time=get_current_time()
    )
    
    # Initialize chatbot_messages and history_state if they are not provided
    allowed_tools = [sum, weather, get_tomorrows_date, user_location, schedule_checkup, get_availability]
    chatbot_messages = chatbot_messages or []
    history_state = history_state or []
    
    # Try to get the AI's reply using the chat_completion
    try:
        assistant_message = chat_completion(message, model="gpt-4", prompt=prompt, messages=history_state, tools=allowed_tools)
    except Exception as e:
        # If an error occurs, raise a Gradio error
        raise gr.Error(e)
        
    # UPDATED
    while(assistant_message.tool_calls):
        for tool_call in assistant_message.tool_calls:
            tool_call_json = json.dumps({ "name": tool_call.function.name, "arguments": tool_call.function.arguments})
            history_state.append({"role": assistant_message.role, "content": tool_call_json})
            
        for tool_call in assistant_message.tool_calls:
            results = call_func(tool_call.function.name, tool_call.function.arguments)
            results_str = str(results) # convert to string
            history_state.append({"role": "function", "tool_call_id": tool_call.id, "name": tool_call.function.name, "content": results_str})

        # Try to get the AI's reply using the get_ai_reply function
        try:
            assistant_message = chat_completion(None, model="gpt-4", prompt=prompt, messages=history_state, tools=allowed_tools)
        except Exception as e:
            # If an error occurs, raise a Gradio error
            raise gr.Error(e)
        
    # Append the user's message and the AI's reply to the chatbot_messages list
    if(assistant_message.content):
        chatbot_messages.append((message, assistant_message.content.strip()))
    else:
        chatbot_messages.append((message, None))
    
    # Append the user's message and the AI's reply to the history_state list
    history_state.append({"role": "user", "content": message})
    history_state.append({"role": "assistant", "content": assistant_message.content})
    
    # Return None (empty out the user's message textbox), the updated chatbot_messages, and the updated history_state
    return None, chatbot_messages, history_state

In [19]:
import gradio as gr
import pandas as pd

def get_availability_df():
    # Get availability data as a list of dictionaries
    availability_data = get_availability()
    
    # Convert the list of dictionaries to a DataFrame
    df = pd.DataFrame(availability_data)
    
    return df 

def get_booked_appointments_df():
    # Get booked appointments data as a list of dictionaries
    appointments_data = get_booked_appointments()
    
    # Convert the list of dictionaries to a DataFrame
    df = pd.DataFrame(appointments_data)
    
    return df

# Define a function to return a chatbot app using Gradio
def get_chatbot_app(share=False):
    # Create the Gradio interface using the Blocks layout
    with gr.Blocks() as app:
        with gr.Tab("Conversation"):
            with gr.Row():
                with gr.Column():
                    # Create a chatbot interface for the conversation
                    chatbot = gr.Chatbot(label="Conversation")
                    # Create a textbox for the user's message
                    message = gr.Textbox(label="Message")
                    # Create a state object to store the conversation history
                    history_state = gr.State()
                    # Create a state object to store the available tools
                    tools = gr.State()
                    # Create a button to send the user's message
                    btn = gr.Button(value="Send")
                btn.click(chat, inputs=[message, chatbot, history_state], outputs=[message, chatbot, history_state])
        with gr.Tab("(Admin) Appointments"):
            refresh = gr.Button(value="Refresh")
            appointments = gr.Dataframe(value=get_booked_appointments_df())
            refresh.click(get_booked_appointments_df, outputs=appointments)
        with gr.Tab("(Admin) Update Appointment"):
            appointment_slots = gr.Dataframe(value=get_availability_df())
        # Return the app
        return app
    
# Call the launch_chatbot function to start the chatbot interface using Gradio
# Set the share parameter to False, meaning the interface will not be publicly accessible
get_chatbot_app().launch()

Running on local URL:  http://127.0.0.1:7866

To create a public link, set `share=True` in `launch()`.


messages: [{'role': 'system', 'content': "\n    You are a helpful AI assistant named Bob created by Valley Day & Night Clinic in Brownsville, TX.\n\n    Today's date: May 08, 2024\n    Current time: 05/08/2024 02:44 PM\n    \n    ## Task\n    \n    Your job is to provide an amazing customer service experience for potential customers of Day & Night Clinic and\n    current customers.\n    \n    Additionally, you are renowned for converting leads into customers.\n    \n    As an AI assistant, you will help faciliate 'check-ups' for Valley Day & Night Clinic's services.\n    \n    ## Check-Up Services\n    \n    Standard - copay of $40 per person\n    \n    A $100 charge will be deducted upon not showing up to the walk-in/appointment at all, so let the customer know about this detail.\n    \n    ## Walk-ins\n    \n    Any person can choose to walk into the clinic during our work hours from 8 A.M. to 5 P.M. and pay the $40 copay.\n    \n    ## Prices\n    \n    Prices for a standard check-u

messages: [{'role': 'system', 'content': "\n    You are a helpful AI assistant named Bob created by Valley Day & Night Clinic in Brownsville, TX.\n\n    Today's date: May 08, 2024\n    Current time: 05/08/2024 06:44 PM\n    \n    ## Task\n    \n    Your job is to provide an amazing customer service experience for potential customers of Day & Night Clinic and\n    current customers.\n    \n    Additionally, you are renowned for converting leads into customers.\n    \n    As an AI assistant, you will help faciliate 'check-ups' for Valley Day & Night Clinic's services.\n    \n    ## Check-Up Services\n    \n    Standard - copay of $40 per person\n    \n    A $100 charge will be deducted upon not showing up to the walk-in/appointment at all, so let the customer know about this detail.\n    \n    ## Walk-ins\n    \n    Any person can choose to walk into the clinic during our work hours from 8 A.M. to 5 P.M. and pay the $40 copay.\n    \n    ## Prices\n    \n    Prices for a standard check-u

messages: [{'role': 'system', 'content': "\n    You are a helpful AI assistant named Bob created by Valley Day & Night Clinic in Brownsville, TX.\n\n    Today's date: May 08, 2024\n    Current time: 05/08/2024 06:44 PM\n    \n    ## Task\n    \n    Your job is to provide an amazing customer service experience for potential customers of Day & Night Clinic and\n    current customers.\n    \n    Additionally, you are renowned for converting leads into customers.\n    \n    As an AI assistant, you will help faciliate 'check-ups' for Valley Day & Night Clinic's services.\n    \n    ## Check-Up Services\n    \n    Standard - copay of $40 per person\n    \n    A $100 charge will be deducted upon not showing up to the walk-in/appointment at all, so let the customer know about this detail.\n    \n    ## Walk-ins\n    \n    Any person can choose to walk into the clinic during our work hours from 8 A.M. to 5 P.M. and pay the $40 copay.\n    \n    ## Prices\n    \n    Prices for a standard check-u

Status code 403 from https://nominatim.openstreetmap.org/search: ERROR - 403 Client Error: Forbidden for url: https://nominatim.openstreetmap.org/search?q=&format=jsonv2&addressdetails=1&limit=1


...

<[ERROR - 403 Client Error: Forbidden for url: https://nominatim.openstreetmap.org/search?q=5524+Whisperwind+Way+Brownsville+Texas&format=jsonv2&addressdetails=1&limit=1] Osm - Geocode [empty]>
None
messages: [{'role': 'system', 'content': "\n    You are a helpful AI assistant named Bob created by Valley Day & Night Clinic in Brownsville, TX.\n\n    Today's date: May 08, 2024\n    Current time: 05/08/2024 06:45 PM\n    \n    ## Task\n    \n    Your job is to provide an amazing customer service experience for potential customers of Day & Night Clinic and\n    current customers.\n    \n    Additionally, you are renowned for converting leads into customers.\n    \n    As an AI assistant, you will help faciliate 'check-ups' for Valley Day & Night Clinic's services.\n    \n    ## Check-Up Services\n    \n    Standard - copay of $40 per person\n    \n    A $100 charge will be deducted upon not showing up to the walk-in/appointment at all, so let the customer know about this detail.\n    \n  

messages: [{'role': 'system', 'content': "\n    You are a helpful AI assistant named Bob created by Valley Day & Night Clinic in Brownsville, TX.\n\n    Today's date: May 08, 2024\n    Current time: 05/08/2024 06:45 PM\n    \n    ## Task\n    \n    Your job is to provide an amazing customer service experience for potential customers of Day & Night Clinic and\n    current customers.\n    \n    Additionally, you are renowned for converting leads into customers.\n    \n    As an AI assistant, you will help faciliate 'check-ups' for Valley Day & Night Clinic's services.\n    \n    ## Check-Up Services\n    \n    Standard - copay of $40 per person\n    \n    A $100 charge will be deducted upon not showing up to the walk-in/appointment at all, so let the customer know about this detail.\n    \n    ## Walk-ins\n    \n    Any person can choose to walk into the clinic during our work hours from 8 A.M. to 5 P.M. and pay the $40 copay.\n    \n    ## Prices\n    \n    Prices for a standard check-u

Status code 403 from https://nominatim.openstreetmap.org/search: ERROR - 403 Client Error: Forbidden for url: https://nominatim.openstreetmap.org/search?q=5524+Whisperwind+Way%2C+Brownsville%2C+TX&format=jsonv2&addressdetails=1&limit=1


<[ERROR - 403 Client Error: Forbidden for url: https://nominatim.openstreetmap.org/search?q=5524+Whisperwind+Way%2C+Brownsville%2C+TX&format=jsonv2&addressdetails=1&limit=1] Osm - Geocode [empty]>
None
messages: [{'role': 'system', 'content': "\n    You are a helpful AI assistant named Bob created by Valley Day & Night Clinic in Brownsville, TX.\n\n    Today's date: May 08, 2024\n    Current time: 05/08/2024 06:45 PM\n    \n    ## Task\n    \n    Your job is to provide an amazing customer service experience for potential customers of Day & Night Clinic and\n    current customers.\n    \n    Additionally, you are renowned for converting leads into customers.\n    \n    As an AI assistant, you will help faciliate 'check-ups' for Valley Day & Night Clinic's services.\n    \n    ## Check-Up Services\n    \n    Standard - copay of $40 per person\n    \n    A $100 charge will be deducted upon not showing up to the walk-in/appointment at all, so let the customer know about this detail.\n    \

messages: [{'role': 'system', 'content': "\n    You are a helpful AI assistant named Bob created by Valley Day & Night Clinic in Brownsville, TX.\n\n    Today's date: May 08, 2024\n    Current time: 05/08/2024 06:46 PM\n    \n    ## Task\n    \n    Your job is to provide an amazing customer service experience for potential customers of Day & Night Clinic and\n    current customers.\n    \n    Additionally, you are renowned for converting leads into customers.\n    \n    As an AI assistant, you will help faciliate 'check-ups' for Valley Day & Night Clinic's services.\n    \n    ## Check-Up Services\n    \n    Standard - copay of $40 per person\n    \n    A $100 charge will be deducted upon not showing up to the walk-in/appointment at all, so let the customer know about this detail.\n    \n    ## Walk-ins\n    \n    Any person can choose to walk into the clinic during our work hours from 8 A.M. to 5 P.M. and pay the $40 copay.\n    \n    ## Prices\n    \n    Prices for a standard check-u